
# Метрики качества классификации

Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [97]:
import pandas 
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

In [98]:
data = pandas.read_csv('classification.csv')

In [99]:
data.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


Заполните таблицу ошибок классификации

In [100]:
classification_Matrix = np.array([[0,0],[0,0]])

In [101]:
for dt in data.values:
    if dt[0] == 1:
        if dt[1] == 1: classification_Matrix[0][0]+=1
        else: classification_Matrix[1][0]+=1
    else:
        if dt[1] == 1: classification_Matrix[0][1]+=1
        else: classification_Matrix[1][1] += 1          

In [80]:
classification_Matrix

array([[43, 34],
       [59, 64]])

In [81]:
resulting_string = " ".join(map(lambda x: str(x[0])+' '+str(x[1]), classification_Matrix))

file_answer = open("answer1.txt", "w")
file_answer.write(resulting_string)
file_answer.close()

Посчитайте основные метрики качества классификатора:

In [82]:
sklearn.metrics.accuracy_score(y_true=data.true, y_pred=data.pred)

0.535

In [102]:
accuracy_values=[]
accuracy_values.append(round(accuracy_score(y_true=data.true, y_pred=data.pred),2))
accuracy_values.append(round(precision_score(y_true=data.true, y_pred=data.pred),2))
accuracy_values.append(round(recall_score(y_true=data.true, y_pred=data.pred),2))
accuracy_values.append(round(f1_score(y_true=data.true, y_pred=data.pred),2))

In [104]:
accuracy_values

[0.54, 0.56, 0.42, 0.48]

Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

для логистической регрессии — вероятность положительного класса (колонка score_logreg),
для SVM — отступ от разделяющей поверхности (колонка score_svm),
для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
для решающего дерева — доля положительных объектов в листе (колонка score_tree).
Загрузите этот файл.

In [105]:
data2 = pandas.read_csv('scores.csv')

In [106]:
data2.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [115]:
accuracy_values2=[]
accuracy_values2.append(roc_auc_score(y_true=data2.true, y_score=data2['score_logreg']))
accuracy_values2.append(roc_auc_score(y_true=data2.true, y_score=data2['score_svm']))
accuracy_values2.append(roc_auc_score(y_true=data2.true, y_score=data2['score_knn']))
accuracy_values2.append(roc_auc_score(y_true=data2.true, y_score=data2['score_tree']))

In [116]:
accuracy_values2

[0.719187675070028, 0.7086834733893557, 0.6351540616246498, 0.6919267707082833]

In [109]:
resulting_string = 'score_svm'

file_answer = open("answer3.txt", "w")
file_answer.write(resulting_string)
file_answer.close()

 Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

In [91]:
#precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

In [122]:
def CalculateThresholds(y_true, y_scores):
    precision, recall, thresholds=precision_recall_curve(y_true,y_scores)
    #для удобства создаем датафрейм
    dt=pandas.DataFrame(data=precision, columns=['precision'])
    #добавляем колонку
    dt['recall']=recall
    #новая колонка, выкидываем записи из точности
    #по фильтру полноты
    dt['precision_sel']=dt.apply(lambda x: 0 if x['recall']<0.7 else x['precision'],axis=1)
    #сортируем по убыванию
    dt.sort_values(['precision_sel'], ascending=[False],inplace=True)
    #переиндексовка
    dt=dt.reset_index(drop=True)
    return dt.loc[0, 'precision_sel']

In [124]:
y_true=data2['true'].values
accuracy_values3={}
accuracy_values3['score_logreg']=CalculateThresholds(y_true,data2['score_logreg'].values)
accuracy_values3['score_svm']=CalculateThresholds(y_true,data2['score_svm'].values)
accuracy_values3['score_knn']=CalculateThresholds(y_true,data2['score_knn'].values)
accuracy_values3['score_tree']=CalculateThresholds(y_true,data2['score_tree'].values)


In [125]:
accuracy_values3

{'score_logreg': 0.6302521008403361,
 'score_svm': 0.6228070175438597,
 'score_knn': 0.6065573770491803,
 'score_tree': 0.6517857142857143}

In [126]:
resulting_string = 'score_tree'

file_answer = open("answer4.txt", "w")
file_answer.write(resulting_string)
file_answer.close()